In [28]:
import langchain
from langchain_community.document_loaders import PyPDFLoader
FILEPATH = r"/Users/rutvik/Developer/DS/Medical-AI-Chatbot/data/Gale Encyclopedia of Medicine. Vol. 1. 2nd Edition ( PDFDrive ).pdf"
def document_loader(PATH):
    loader = PyPDFLoader(FILEPATH)
    document = loader.load()
    return document

doc = document_loader(PATH=FILEPATH)

        
        

In [29]:
docs = doc[:50]

In [30]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
)
splitted_doc = text_splitter.split_documents(docs)

In [31]:
import getpass
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()
os.getenv("GOOGLE_API_KEY")

from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_chroma import Chroma

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)



In [60]:

vectorstore = Chroma.from_documents(documents=splitted_doc, embedding=embeddings)

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 10})

In [61]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)


In [68]:
"what is Definition of Acetaminophen?"result = rag_chain.invoke({"input": })
print(result['answer'])

In [69]:
print(result['answer'])

Acetaminophen is a medicine used to relieve pain and reduce fever. It is as effective as aspirin for reducing fever but is less likely to irritate the stomach. However, unlike aspirin, it does not reduce redness, stiffness, or swelling associated with arthritis. 



In [70]:
result

{'input': 'what is Definition of Acetaminophen?',
 'context': [Document(metadata={'page': 32, 'source': '/Users/rutvik/Developer/DS/Medical-AI-Chatbot/data/Gale Encyclopedia of Medicine. Vol. 1. 2nd Edition ( PDFDrive ).pdf'}, page_content='and reduces fever about as well as aspirin. But differencesbetween these two common drugs exist. Acetaminophenis less likely than aspirin to irritate the stomach. However,unlike aspirin, acetaminophen does not reduce the red-ness, stiffness, or swelling that accompany arthritis.\nRecommended dosage\nThe usual dosage for adults and children age 12 and'),
  Document(metadata={'page': 32, 'source': '/Users/rutvik/Developer/DS/Medical-AI-Chatbot/data/Gale Encyclopedia of Medicine. Vol. 1. 2nd Edition ( PDFDrive ).pdf'}, page_content='and reduces fever about as well as aspirin. But differencesbetween these two common drugs exist. Acetaminophenis less likely than aspirin to irritate the stomach. However,unlike aspirin, acetaminophen does not reduce the re